# I3M Computation by 3R



This project will perform the mask_RCNN inference of MT in order to eventually compute I3M

In [1]:
#@title Configuration { run: "auto" }
run_mode = "Comparaison inference / manual" #@param ["Inference", "Comparaison inference / manual"]
endpoints_detection_method = "Gradient" #@param ["Ray tracing", "Skeleton", "Ray tracing + skeleton", "Gradient"]
HD = "Image already HD" #@param ["Apply HD filter", "Image already HD"]
debug = True #@param {type: "boolean" }

## Select Model + Image

In [3]:
!pwd

/content


In [4]:
!rm /content/*.jpg || {}

from google.colab import files

print('please select your Images and/or Manual segmentations')
uploaded = files.upload()
filenames = list(uploaded.keys())
image_filenames = [filename for filename in filenames if filename.endswith('.jpg') or filename.endswith('.png')]
manual_segmentation_filenames = [filename for filename in filenames if filename.endswith('.json')]

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

rm: cannot remove '/content/*.jpg': No such file or directory
/bin/bash: {}: command not found
please select your Images and/or Manual segmentations


Saving 006_r.jpg to 006_r.jpg
Saving 006_r.json to 006_r.json
Saving 008_l.jpg to 008_l.jpg
Saving 008_l.json to 008_l.json
Saving 021_r.jpg to 021_r.jpg
Saving 021_r.json to 021_r.json
Saving 022_r.jpg to 022_r.jpg
Saving 022_r.json to 022_r.json
Saving 024_l.jpg to 024_l.jpg
Saving 024_l.json to 024_l.json
Saving 026_l.jpg to 026_l.jpg
Saving 026_l.json to 026_l.json
Saving 027_r.jpg to 027_r.jpg
Saving 027_r.json to 027_r.json
Saving 029_l.jpg to 029_l.jpg
Saving 029_l.json to 029_l.json
Saving 036_r.jpg to 036_r.jpg
Saving 036_r.json to 036_r.json
Saving 038_r.jpg to 038_r.jpg
Saving 038_r.json to 038_r.json
Saving 043_l.jpg to 043_l.jpg
Saving 043_l.json to 043_l.json
Saving 055_r.jpg to 055_r.jpg
Saving 055_r.json to 055_r.json
Saving 066_r.jpg to 066_r.jpg
Saving 066_r.json to 066_r.json
Saving 070_r.jpg to 070_r.jpg
Saving 070_r.json to 070_r.json
Saving 074_l.jpg to 074_l.jpg
Saving 074_l.json to 074_l.json
Saving 085_l.jpg to 085_l.jpg
Saving 085_l.json to 085_l.json
Saving 0

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [6]:
downloaded = drive.CreateFile({'id':"10jP0flXeozBQUXcx6Q7CUonaQiTZAjrV"})  # I3M - JUNE 2021
#downloaded = drive.CreateFile({'id':"1CAn1PBg_OrGhN41LP51UNY46zTsi3xo-"})  # mask_rcnn_tooth_segmentation_0006
#downloaded = drive.CreateFile({'id':"19euYX5zo1ty5HacZ0mwXuUoHygFaWLkV"}) # mask_rcnn_tooth_segmentation_0008
#downloaded = drive.CreateFile({'id':"1o6uigkw3ZSN2H1vBRv5-uqLX3UiD7m-w"}) # mask_rcnn_tooth_segmentation_0018
#downloaded = drive.CreateFile({'id':"1eFsvwd7knZBOpUTO_7iwIcipnvE7Cq1W"}) # mask_rcnn_tooth_segmentation_0019

downloaded.GetContentFile('I3M.h5')  

## Install backbone in Google Colab





In [7]:
#%tensorflow_version 1.x

In [8]:
!git clone https://github.com/matterport/Mask_RCNN
%cd Mask_RCNN
!pip3 install -r requirements.txt
!python3 setup.py install

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 33.80 MiB/s, done.
Resolving deltas: 100% (562/562), done.
/content/Mask_RCNN
     |████████████████████████████████| 163kB 5.2MB/s 
     |████████████████████████████████| 204kB 8.9MB/s 
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in fu

In [9]:
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ../../

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 25.82 MiB/s, done.
Resolving deltas: 100% (576/576), done.
/content/Mask_RCNN/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/Mask_RCNN/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.

In [10]:
!git clone https://www.github.com/iozzinor/Apex_points
!pip3 install scikit-image==0.18.1

Cloning into 'Apex_points'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 87 (delta 43), reused 70 (delta 26), pack-reused 0
Unpacking objects: 100% (87/87), done.
     |████████████████████████████████| 29.2MB 106kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
      Successfully uninstalled scikit-image-0.16.2


In [11]:
!pip3 uninstall -y keras-nightly
!pip3 uninstall -y keras
!pip3 install keras==2.2.5

#!pip3 uninstall -y tensorflow
!pip3 install tensorflow==1.15.0rc2

!pip3 install h5py==2.10.0

Uninstalling keras-nightly-2.5.0.dev2021032900:
  Successfully uninstalled keras-nightly-2.5.0.dev2021032900
Uninstalling Keras-2.4.3:
  Successfully uninstalled Keras-2.4.3
     |████████████████████████████████| 337kB 5.2MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 412.3MB 33kB/s 
     |████████████████████████████████| 3.8MB 24.5MB/s 
     |████████████████████████████████| 512kB 36.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=70d4d7c9fd8318be778856ab20f6696585f210863c87926656dfd9485713267f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0rc2 which is incompatible.
  Found existing installation: tensorboard

## Import and Config

In [12]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

import Apex_points
from Apex_points.ray_tracing import localize_apices_engine as ray_tracing_engine
from Apex_points.skeleton import localize_apices_engine as skeleton_engine
from Apex_points.gradient import localize_apices_engine as gradient_engine

import traceback

Using TensorFlow backend.


In [13]:
## UPDATE WT
class RadioConfig(Config):
    # Give the configuration a recognizable name
    NAME = "tooth_segmentation"
    
    NUM_CLASSES = 1 + 1 #NB : 2 because MT and JAD

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    STEPS_PER_EPOCH = 250

    # New
    IMAGE_MAX_DIM = 256
    IMAGE_MIN_DIM = 256

config = RadioConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

## Inference production

In [14]:
%cd ..

/content


In [15]:
MODEL_DIR = ""
class InferenceConfig(RadioConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = "I3M.h5"
#model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead

Instructions for updating:
Use `tf.cast` instead.
Loading weights from  I3M.h5


In [16]:
import numpy as np
from skimage.morphology import skeletonize
from skimage import exposure
from PIL import Image, ImageDraw


cnames = ['BG', 'MT']

In [17]:
# Romain - add on : create dict
#
# Regis - edit: store results in the global variable as dictionnaries. dict_res is no longer a list but a dict, whose keys are the radio UNIQUE identifiers
# Advantages: no longer need to maintain two arrays side by side ('Radio_ID' and 'I3M') with the need to have same length and order
dict_res = {}

def adaptHist(path_image): 
    img_to_pred_arr = np.array(Image.open(path_image).convert('RGB'))
    img_to_pred_adapthist = exposure.equalize_adapthist(img_to_pred_arr/np.max(img_to_pred_arr))
    img_to_pred_adapthist = Image.fromarray((img_to_pred_adapthist * 255).astype(np.uint8))
    return img_to_pred_adapthist


def make_predictions(image_paths):
  """
  Returns
  -------
    The result of the function `make_visualisations`
  """
  #images = [Image.open(image_path).convert('RGB') for image_path in image_paths]
  #names = [os.path.splitext(os.path.basename(filename))[0] for filename in image_paths]
  #np_images = [np.array(image) for image in images]
  
  if(HD=="Image already HD"):
    images = [Image.open(image_path).convert('RGB') for image_path in image_paths]
  else: 
    images = [adaptHist(image_path) for image_path in image_paths]

  names = [os.path.splitext(os.path.basename(filename))[0] for filename in image_paths]
  np_images = [np.array(image) for image in images]
  
  results = [model.detect([image], verbose=1) for image in np_images]
  return make_visualizations(results, images, names)

def make_visualizations(results, images, names):
   """
   Returns
   -------
   An array of the results returned by `make_prediction_image`
   """
   return [make_prediction_image(r[0], i, name) for r, i, name in zip(results, images, names)]

def make_prediction_image(r, img_to_pred, image_name, dict_res=dict_res):
  """
  Returns
  -------
    None in case of failure.
    Else: a tupple of :
    - the output image, as a PILLOW object.
    - the prediction engine result
    - the model result
    - an array of the mineralized tissue masks
  """
  # masks has a shape (height, width, number of detected classes)
  masks = r['masks']
  # masks is now an array with 'number of detected classes' elements
  # each element has a shape (height, width)
  masks_count = masks.shape[-1]
  masks = [mask.reshape((config.IMAGE_MAX_DIM, config.IMAGE_MAX_DIM)) for mask in np.split(masks, masks_count, axis=-1)]
  class_ids = r['class_ids']
  mt_masks = []
  for i, mask in enumerate(masks):
      current_class_name = cnames[class_ids[i]]
      if current_class_name == 'MT':
          mt_masks.append(mask)

  apices_result = apex_prediction_image(img_to_pred, mt_masks, image_name)
  if apices_result is not None:
    dict_res[image_name] = {'I3M': apices_result['I3M']}
    return apices_result['output_image'], apices_result, r, mt_masks
  return None
            
def apex_prediction_image(img_to_pred, mt_masks, image_name):
    if endpoints_detection_method == 'Ray tracing':
        return Apex_points.localize_apices(img_to_pred, mt_masks, ray_tracing_engine, debug, '.', image_name)
    elif endpoints_detection_method == 'Skeleton':
        return Apex_points.localize_apices(img_to_pred, mt_masks, skeleton_engine, debug, '.', image_name)
    elif endpoints_detection_method == 'Ray tracing + skeleton':
        width, height = img_to_pred.size
        result_image = Image.new('RGB', (width * 2, height))
        global_result = { 'I3M': -1 }
        apices_result = Apex_points.localize_apices(img_to_pred, mt_masks, ray_tracing_engine, debug, '.', image_name)
        if apices_result is not None:
            result_image.paste(apices_result['output_image'].resize((width, height)), (0, 0))
            global_result['I3M'] = apices_result['I3M']
        apices_result = Apex_points.localize_apices(img_to_pred, mt_masks, skeleton_engine, debug, '.', image_name)
        if apices_result is not None:
            result_image.paste(apices_result['output_image'].resize((width, height)), (width, 0))
        global_result['output_image'] = result_image
        return global_result
    elif endpoints_detection_method == 'Gradient':
      return Apex_points.localize_apices(img_to_pred, mt_masks, gradient_engine, debug, '.', image_name)



In [18]:
image_paths = image_filenames

In [ ]:
%cd /content
result_predictions = make_predictions(image_filenames)
result_images = [result[0] for result in result_predictions]

/content
YES
Processing 1 images
image                    shape: (256, 256, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 256, 256, 3)      min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  256.00000  int64
anchors                  shape: (1, 16368, 4)         min:   -1.41976  max:    2.16878  float32
Processing 1 images
image                    shape: (256, 256, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 256, 256, 3)      min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  256.00000  int64
anchors                  shape: (1, 16368, 4)         min:   -1.41976  max:    2.16878  float32
Processing 1 images
image                    shape: (256, 256, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 256, 256, 3)      min: -123.70000  ma

In [ ]:
# save the results
![ -d Predictions ] || mkdir Predictions
for i, result_image in enumerate(result_images):
  if result_image is None:
    print(f'image at index {i} is None')
    continue
  try: 
    filename = image_filenames[i]
    image_name, image_extension = os.path.splitext(filename) # get the image name from the filename
    image_extension = image_extension[1:] # remove the leading dot

    output_path = os.path.join('Predictions', f'{image_name}-pred-{endpoints_detection_method}.{image_extension}')
    result_image.save(output_path, quality=100, subsampling=0)
    print(f'saved result at {output_path}')
  except Exception as exception:
    print(f'error in saving: {exception}')
    print(traceback.format_exc())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# show at max 4 images
def _show_images(images=None):
  if images is None:
    images = result_images
  images_to_show = [image for image in images if image is not None][:4]
  grid_size = 1 if len(images_to_show) < 2 else 2
  fig, _ = plt.subplots(ncols=grid_size, nrows=grid_size)
  for i, image in enumerate(images_to_show):
      fig.axes[i].imshow(image)
  plt.show()

In [ ]:
if debug:
  _show_images()

# Comparaison inference / manual (if applicable: see `run_mode`)

In [ ]:
%matplotlib inline
# Regis - add on: compute area accuracy

import logging
import json

def _create_merged_mask(mt_masks):
  # create the composite image (logical or of all masks)
  mt_mask = np.zeros_like(mt_masks[0])
  for mask in mt_masks:
    mt_mask |= mask
  return mt_mask

def _shapes_to_masks(shapes, width, height):
  masks = []
  for shape in shapes:
    mask = Image.new('L', (width, height))
    points = [tuple(point) for point in shape['points']]
    ImageDraw.Draw(mask).polygon(points, fill=1)
    masks.append(np.array(mask))
  return masks

  masks = np.zeros((height, width))

def _json_file_to_mt_mask(json_filename):
    with open(json_filename) as json_file:
      annotations = json.load(json_file)
    shapes = annotations['shapes']
    mt_shapes = [shape for shape in shapes if shape['label'] == 'MT']
    mt_masks = _shapes_to_masks(mt_shapes, annotations['imageWidth'], annotations['imageHeight'])
    return _create_merged_mask(mt_masks)

def _number_pixels_in_mask(mask):
  """
  Parameters
  ----------
  mask: np.array
    A binary mask (or an int mask: any strict positive value will be considered as True).
  """
  return len(list(zip(*np.where(mask>0))))

def _compare_inference_against_manual():
  # mt masks is the third key of the prediction tupple
  merged_masks = [_create_merged_mask(result_prediction[3]) for result_prediction in result_predictions]
  _show_images(merged_masks)

  # load the manual segmentations
  # can zip image_filenames and result_predictions because they are assumed to have the same length (result_predictions is instantiated using the image_filenames array as the parameter of the function `make_predictions`)
  for image_filename, prediction_mt_mask, result_prediction in zip(image_filenames, merged_masks, result_predictions):
    image_basename = image_filename.replace('.jpg', '').replace('.png', '')
    json_filename = '{}.json'.format(image_basename)
    if not os.path.isfile(json_filename):
      logging.warning('ignoring missing manual segmentation file: {}'.format(json_filename))
      continue
    ground_truth_mt_mask = _json_file_to_mt_mask(json_filename)

    union = ground_truth_mt_mask | prediction_mt_mask
    intersection = ground_truth_mt_mask & prediction_mt_mask

    tp_mask = intersection
    tn_mask = np.where(union>0,0,1)
    fp_mask = union & np.where(ground_truth_mt_mask>0,0,1)
    fn_mask = union & np.where(prediction_mt_mask>0,0,1)

    accuracy_image = Image.new('RGB', (union.shape[1], union.shape[0]))
    # TODO: refactoriser bibliothèque Apex_points pour ne plus appeler la fonction "privée" Apex_points.utils._apply_mask_to_image.
    Apex_points.utils._apply_mask_to_image(accuracy_image, tp_mask, (0, 255, 0), 1) # true positives: green
    Apex_points.utils._apply_mask_to_image(accuracy_image, tn_mask, (0, 0, 0), 1) # true negatives: black
    Apex_points.utils._apply_mask_to_image(accuracy_image, fn_mask, (255, 0, 0), 1) # false negatives (missing pixels): red
    Apex_points.utils._apply_mask_to_image(accuracy_image, fp_mask, (255, 128, 0), 1) # false positives: orange

    stat_masks = {'TP': tp_mask, 'TN': tn_mask, 'FP': fp_mask, 'FN': fn_mask}

    for (key, mask) in stat_masks.items():
      dict_res[image_basename][key] = _number_pixels_in_mask(mask)

    if debug:
      plt.imshow(accuracy_image)


In [ ]:
if run_mode == 'Comparaison inference / manual':
  _compare_inference_against_manual()

In [ ]:
# Romain - add on : write csv results
import csv

keys = ['I3M', 'TP', 'TN', 'FP', 'FN'] if run_mode == 'Comparaison inference / manual' else ['I3M']

dict_res_file = open("./Predictions/results.csv", "w")
writer = csv.writer(dict_res_file)
writer.writerow(['Radio_ID'] + keys)
for radio_id in sorted(dict_res.keys()):
  values = [dict_res[radio_id][key] if key in dict_res[radio_id] else '' for key in keys]
  row = [radio_id]
  row.extend(values)
  writer.writerow(row)
dict_res_file.close()

  ## Download final file

In [ ]:
!zip -r Predictions.zip Predictions/

In [ ]:
files.download('Predictions.zip')

Last step: right click on Predictions.zip then download